In this analysis, we will use gower distance (https://pypi.org/project/gower/) to build a simple food replacement recommender. In the absence of a particular food, the recommender will tell us which is the food with most similar
nutriotional properties. <br>
Here, we will use the cleaned up dataset generated from the original one.

# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import datetime
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import random as rand

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import DistanceMetric
import gower

# Load data

In [3]:
#food_data = pd.read_excel('data/MyFoodData Nutrition Facts SpreadSheet Release 1.4.xlsx')
food_data = pd.read_excel('data/MyFoodData Nutrition Facts Redux.xlsx')

In [4]:
food_data.head(3)

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg),Serving Weight 1 (g),Serving Description 1 (g),Serving Weight 2 (g),Serving Description 2 (g),Serving Weight 3 (g),Serving Description 3 (g),Serving Weight 4 (g),Serving Description 4 (g),Serving Weight 5 (g),Serving Description 5 (g),Serving Weight 6 (g),Serving Description 6 (g),Serving Weight 7 (g),Serving Description 7 (g),Serving Weight 8 (g),Serving Description 8 (g),Serving Weight 9 (g),Serving Description 9 (g),200 Calorie Weight (g)
0,Hearts Of Palm Raw,Vegetables,115,0.20,2.7,25.61,17.16,1.5,0.0,0.046,18,1.69,1806,10,68.0,3.0,8.0,0.0,0.0,0.50,NaN,24.11,69.50,13.0,76.0,-32.101,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,14,3.73,0.644,NaN,0.7,NaN,NaN,NaN,0.050,0.180,0.900,NaN,0.810,NaN,24.0,0.0,24.0,24.0,0.0,NaN,0.0,41.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,5.0,89.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,173.913
1,Asparagus,Vegetables,20,0.12,2.2,3.88,1.88,2.1,0.0,0.040,24,2.14,202,14,756.0,38.0,5.6,0.0,0.0,1.13,NaN,1.78,93.22,10.0,40.0,-1.940,0.0,NaN,NaN,0.23,0.65,1.00,0.0,0.0,0.0,NaN,NaN,52.0,2,0.54,0.189,0.158,2.3,NaN,NaN,NaN,0.143,0.141,0.978,0.274,0.091,NaN,52.0,0.0,52.0,52.0,16.0,0.6,0.0,449.0,9.0,0.0,710.0,NaN,NaN,0.0,41.6,0.0,NaN,0.0,50.0,NaN,0.0,0.0,0.0,27.0,84.0,75.0,128.0,104.0,31.0,31.0,75.0,52.0,115.0,91.0,49.0,115.0,508.0,233.0,93.0,71.0,106.0,NaN,0.0,0.0,0.0,134.0,1 cup,12.0,"1 spear, small (5 inch long or less)",16.0,"1 spear, medium (5-1/4 inch to 7 inch long)",20.0,"1 spear, large (7-1/4 inch to 8-1/2 inch)",24.0,"1 spear, extra large (8-3/4 inch to 10 inch long)",3.5,1 spear tip (2 inch long or less),NaN,NaN,NaN,NaN,NaN,NaN,1000.000
2,Asparagus (Cooked),Vegetables,22,0.22,2.4,4.11,1.30,2.0,0.0,0.048,23,0.91,224,14,1006.0,50.0,7.7,0.0,0.0,1.50,NaN,2.11,92.63,29.0,76.0,-2.220,0.0,NaN,NaN,0.08,0.42,0.79,0.0,0.0,0.0,0.0,NaN,54.0,14,0.60,0.165,0.154,6.1,21.9,NaN,NaN,0.162,0.139,1.084,0.225,0.079,NaN,149.0,0.0,149.0,149.0,26.1,0.9,0.0,604.0,0.0,30.0,771.0,NaN,NaN,0.0,50.6,0.0,NaN,0.0,105.0,NaN,0.0,0.0,0.0,29.0,92.0,82.0,140.0,113.0,34.0,34.0,82.0,57.0,125.0,99.0,53.0,125.0,555.0,255.0,101.0,77.0,116.0,NaN,0.0,0.0,0.0,90.0,1/2 cup,60.0,4 spears (1/2 inch base),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

# Data Preparation

## Remove useless columns

This step will be based on our previous work

In [5]:
serving_cols = food_data.columns[food_data.columns.str.contains('Serving')]
food_data.drop(serving_cols,axis=1,inplace=True)

In [6]:
food_data.drop(['200 Calorie Weight (g)'],axis=1, inplace=True)

## Replace NaNs with 0

In [7]:
food_data.replace(np.nan, 0, inplace=True)

In [8]:
food_data.isna().sum()

Name                   0
Food Group             0
Calories               0
Fat (g)                0
Protein (g)            0
                      ..
Serine (mg)            0
Hydroxyproline (mg)    0
Alcohol (g)            0
Caffeine (mg)          0
Theobromine (mg)       0
Length: 97, dtype: int64

In [9]:
food_data.shape

(420, 97)

In [10]:
food_data.columns

Index(['Name', 'Food Group', 'Calories', 'Fat (g)', 'Protein (g)',
       'Carbohydrate (g)', 'Sugars (g)', 'Fiber (g)', 'Cholesterol (mg)',
       'Saturated Fats (g)', 'Calcium (mg)', 'Iron, Fe (mg)',
       'Potassium, K (mg)', 'Magnesium (mg)', 'Vitamin A, IU (IU)',
       'Vitamin A, RAE (mcg)', 'Vitamin C (mg)', 'Vitamin B-12 (mcg)',
       'Vitamin D (mcg)', 'Vitamin E (Alpha-Tocopherol) (mg)',
       'Added Sugar (g)', 'Net-Carbs (g)', 'Water (g)', 'Omega 3s (mg)',
       'Omega 6s (mg)', 'PRAL score', 'Trans Fatty Acids (g)',
       'Soluble Fiber (g)', 'Insoluble Fiber (g)', 'Sucrose (g)',
       'Glucose (Dextrose) (g)', 'Fructose (g)', 'Lactose (g)', 'Maltose (g)',
       'Galactose (g)', 'Starch (g)', 'Total sugar alcohols (g)',
       'Phosphorus, P (mg)', 'Sodium (mg)', 'Zinc, Zn (mg)', 'Copper, Cu (mg)',
       'Manganese (mg)', 'Selenium, Se (mcg)', 'Fluoride, F (mcg)',
       'Molybdenum (mcg)', 'Chlorine (mg)', 'Thiamin (B1) (mg)',
       'Riboflavin (B2) (mg)', 'Nia

# Food substitute recommender demo

We will use the *gower.topn* function from the gower package. <br>
The function returns the top n matches with respect to a match we pass to the function.

## Select a food

In [11]:
food_data.iloc[1,]

Name                    Asparagus
Food Group             Vegetables
Calories                       20
Fat (g)                      0.12
Protein (g)                   2.2
                          ...    
Serine (mg)                 106.0
Hydroxyproline (mg)           0.0
Alcohol (g)                   0.0
Caffeine (mg)                 0.0
Theobromine (mg)              0.0
Name: 1, Length: 97, dtype: object

## Select all the rest of the foods to pick the best match

In [12]:
food_data.iloc[:,:-1]

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg)
0,Hearts Of Palm Raw,Vegetables,115,0.20,2.70,25.61,17.16,1.5,0.0,0.046,18,1.69,1806,10,68.0,3.0,8.0,0.00,0.0,0.50,0.0,24.11,69.50,13.0,76.0,-32.101,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,140.0,14,3.73,0.644,0.000,0.7,0.0,0.0,0.0,0.050,0.180,0.900,0.000,0.810,0.0,24.0,0.0,24.0,24.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Asparagus,Vegetables,20,0.12,2.20,3.88,1.88,2.1,0.0,0.040,24,2.14,202,14,756.0,38.0,5.6,0.00,0.0,1.13,0.0,1.78,93.22,10.0,40.0,-1.940,0.0,0.0,0.0,0.23,0.65,1.00,0.0,0.00,0.0,0.00,0.0,52.0,2,0.54,0.189,0.158,2.3,0.0,0.0,0.0,0.143,0.141,0.978,0.274,0.091,0.0,52.0,0.0,52.0,52.0,16.0,0.6,0.0,449.0,9.0,0.0,710.0,0.0,0.0,0.0,41.6,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,27.0,84.0,75.0,128.0,104.0,31.0,31.0,75.0,52.0,115.0,91.0,49.0,115.0,508.0,233.0,93.0,71.0,106.0,0.0,0.0,0.0
2,Asparagus (Cooked),Vegetables,22,0.22,2.40,4.11,1.30,2.0,0.0,0.048,23,0.91,224,14,1006.0,50.0,7.7,0.00,0.0,1.50,0.0,2.11,92.63,29.0,76.0,-2.220,0.0,0.0,0.0,0.08,0.42,0.79,0.0,0.00,0.0,0.00,0.0,54.0,14,0.60,0.165,0.154,6.1,21.9,0.0,0.0,0.162,0.139,1.084,0.225,0.079,0.0,149.0,0.0,149.0,149.0,26.1,0.9,0.0,604.0,0.0,30.0,771.0,0.0,0.0,0.0,50.6,0.0,0.0,0.0,105.0,0.0,0.0,0.0,0.0,29.0,92.0,82.0,140.0,113.0,34.0,34.0,82.0,57.0,125.0,99.0,53.0,125.0,555.0,255.0,101.0,77.0,116.0,0.0,0.0,0.0
3,Garden Cress,Vegetables,32,0.70,2.60,5.50,4.40,1.1,0.0,0.023,81,1.30,606,38,6917.0,346.0,69.0,0.00,0.0,0.70,0.0,4.40,89.40,76.0,152.0,-10.753,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,76.0,14,0.23,0.170,0.553,0.9,0.0,0.0,0.0,0.080,0.260,1.000,0.242,0.247,0.0,80.0,0.0,80.0,80.0,19.5,0.2,0.0,4150.0,0.0,0.0,12500.0,0.0,0.0,0.0,541.9,0.0,0.0,239.0,228.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cucumber,Vegetables,15,0.11,0.65,3.63,1.67,0.5,0.0,0.037,16,0.28,147,13,105.0,5.0,2.8,0.00,0.0,0.03,0.0,3.13,95.23,5.0,28.0,-2.445,0.0,0.0,0.0,0.03,0.76,0.87,0.0,0.01,0.0,0.83,0.0,24.0,2,0.20,0.041,0.079,0.3,1.3,0.0,0.0,0.027,0.033,0.098,0.259,0.040,0.0,7.0,0.0,7.0,7.0,6.0,0.1,0.0,45.0,11.0,0.0,23.0,0.0,0.0,0.0,16.4,0.0,0.0,5.0,32.0,0.0,0.0,0.0,0.0,5.0,19.0,21.0,29.0,29.0,6.0,4.0,19.0,11.0,22.0,44.0,10.0,24.0,41.0,196.0,24.0,15.0,20.0,0.0,0.0,0

In [13]:
#food_data.iloc[1:2,2:]

In [14]:
#food_data.iloc[:,2:]

## Look for the best match for 'asparagus' among the rest of the foods based on all the attributes

In [15]:
gower_dist = gower.gower_topn(food_data.iloc[1:2,2:], food_data.iloc[:,2:], n = len(food_data))

The array returned contains the indeces of the best matches ranked. The first index (i.e., 1) is clearly 'asparagus' as
it's the most similar food compared to itself

In [16]:
gower_dist

{'index': array([  1,  69,  49,   2,  39,  56,  66,  44,  79,  19,  45,  34,  25,
         33,  47,  48,  35,  80,  32,  30,   6,   8,  76,  23,  16,  65,
         12,  46,  18,  50,  78,  41,  73,  26,  51,  55,  24,  74,   4,
         60,  81,  54, 259,  43, 271,  42, 237,  61,  58,  21,  29,  82,
        235, 266, 147,   5, 268, 145, 280, 397,  31, 274, 265, 152,  38,
         28, 264,  75,  57,  63, 234, 231, 209, 248, 276, 418, 262, 260,
        251,  36,  77, 277, 255, 246,  71,  17,  83, 210,  52,  59, 148,
        263, 272, 281, 223, 232,  37,   7, 377, 269, 275, 215, 233, 254,
        236, 376, 273, 282, 258, 247,  53, 267, 261, 241,  11, 214, 240,
        242, 270, 244, 226, 257,  84, 256,  72,  64, 419, 379, 208, 393,
        243, 245, 224, 279, 229, 218, 146, 154, 111,  20, 216,  62, 113,
        225, 239, 253, 404,  67,  22, 417, 284, 324, 278, 149, 358, 406,
        348, 360, 361,   3, 375,  68, 153, 373, 374, 222,  40, 177,  96,
        114, 112,  97, 405, 409,  70,   0,

The second index returns cooked cauliflower, which is the most similar food to asparagus

In [17]:
food_data.iloc[69,:]

Name                   Cooked Cauliflower
Food Group                     Vegetables
Calories                               23
Fat (g)                              0.45
Protein (g)                          1.84
                              ...        
Serine (mg)                          96.0
Hydroxyproline (mg)                   0.0
Alcohol (g)                           0.0
Caffeine (mg)                         0.0
Theobromine (mg)                      0.0
Name: 69, Length: 97, dtype: object

# Food substitute recommender function: Basic

## Define a function 

The function will take the food name, and a few other params, to return a set of most similar food ranked by similarity

In [18]:
def food_substitute_recommender(food_name: str, food_data: pd.DataFrame, top_matches: int):
    """
    This function takes a food and returns a list of ranked foods most similar to it
    food_name: the name of the food we want a substitute for
    data: the dataset containing the matches
    top_matches: how many similar food we want returned
    """

    # Look for a particular food in the food data and return a pandas df  
    match = food_data.loc[food_data['Name'].str.contains(food_name,case=False)]
    
    # If the resulting df is empty, this means no matches were found
    if match.empty:
        raise Exception("The food was not found. Try another one")

    # Get the index of the found food as an integer
    index_int = match.index.values.astype(int)[0]
    
    # Calculate the most similar foods by matching our target to the rest of the data
    # Notice that we're not using food name and food group as features to match (food_data.iloc[:,2:])
    # Also, notice that the best match will always be the same food, this means that top_matches will include the
    # food we are looking up. We need to skip this and add an element to the list of returned matches (n = top_matches+1)
    gower_dist = gower.gower_topn(food_data.iloc[index_int:index_int+1,2:], food_data.iloc[:,2:], n = top_matches+1)
    
    # We get the most indeces of the most similar foods starting from 1, so that we exclude the food we were looking for
    most_similar_indeces = gower_dist['index'][1:top_matches+1].astype(int)
    
    print(f"""The {top_matches} most similar foods to {food_name} are: """)
    
    return food_data.filter(items = most_similar_indeces, axis=0).reset_index(drop=True)
    

# Test

Let's select a random food from the dataset and look for the best matches

In [21]:
food_data.sample(1)

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
167,Bacon (Pan-Fried),Meats,468,35.09,33.92,1.7,0.0,0.0,99.0,11.964,11,0.95,499,31,37.0,11.0,0.0,1.09,0.4,0.42,0.0,1.7,23.65,293.0,5695.0,19.49,0.144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,388.0,1684,3.06,0.105,0.022,50.7,0.0,0.0,0.0,0.56,0.232,10.457,1.1,0.535,0.0,0.0,0.0,0.0,0.0,89.5,8.1,11.0,0.0,0.0,0.0,0.0,0.0,0.4,17.0,0.0,0.0,0.0,15530.0,6112.0,223.0,4.0,26.0,8.0,407.0,1504.0,1619.0,2814.0,3041.0,937.0,385.0,1421.0,1355.0,1729.0,2203.0,1391.0,1958.0,3198.0,5223.0,1545.0,1382.0,1428.0,119.0,0.0,0.0,0.0


In [90]:
food_substitute_recommender('Bacon (Pan-Fried)', food_data, 3)

C:\Users\valen\AppData\Local\Temp/ipykernel_11576/953932786.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  match = food_data.loc[food_data['Name'].str.contains(food_name,case=False)]


Exception: The food was not found. Try another one

In [91]:
food_data.sample(1)

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
372,Swiss Cheese,Dairy and Egg Products,393,30.99,26.96,1.44,0.0,0.0,93.0,18.227,890,0.13,72,33,1047.0,288.0,0.0,3.06,0.0,0.6,0.0,1.44,37.63,156.0,1295.0,20.491,0.987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,574.0,187,4.37,0.047,0.026,30.0,0.0,0.0,0.0,0.011,0.302,0.064,0.353,0.071,0.0,10.0,0.0,10.0,10.0,13.8,0.1,283.0,61.0,1.0,1.0,8.0,0.0,0.0,0.0,1.4,0.0,6.3,8046.0,1341.0,125.0,9.0,18.0,0.0,401.0,1038.0,1537.0,2959.0,2585.0,784.0,290.0,1662.0,1693.0,2139.0,927.0,1065.0,914.0,1569.0,5704.0,508.0,3690.0,1640.0,0.0,0.0,0.0,0.0


In [93]:
food_substitute_recommender('Swiss Cheese',food_data,5)

The 5 most similar foods to Swiss Cheese are: 


,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
0,Gruyere Cheese,Dairy and Egg Products,413,32.34,29.81,0.36,0.36,0.0,110.0,18.913,1011,0.17,81,36,948.0,271.0,0.0,1.60,0.6,0.28,0.0,0.36,33.19,433.0,1300.0,21.193,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,605.0,714,3.90,0.032,0.017,14.5,0.0,0.0,0.0,0.060,0.279,0.106,0.562,0.081,0.0,10.0,0.0,10.0,10.0,15.4,0.0,268.0,33.0,0.0,0.0,0.0,0.0,0.6,24.0,2.7,0.0,0.0,10043.0,1733.0,0.0,0.0,0.0,0.0,421.0,1089.0,1612.0,3102.0,2710.0,822.0,304.0,1743.0,1776.0,2243.0,972.0,1117.0,958.0,1645.0,5981.0,533.0,3869.0,1719.0,0.0,0.0,0.0,0.0
1,Grated Parmesan,Dairy and Egg Products,420,27.84,28.42,13.91,0.07,0.0,86.0,15.371,853,0.49,180,34,974.0,262.0,0.0,1.40,0.5,0.53,0.0,13.91,22.65,119.0,1352.0,21.344,0.876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,627.0,1804,4.20,0.040,0.071,34.4,0.0,0.0,0.0,0.026,0.358,0.080,0.450,0.081,0.0,6.0,0.0,6.0,6.0,14.1,0.2,255.0,73.0,0.0,0.0,0.0,0.0,0.5,21.0,1.7,0.0,7.1,7130.0,1386.0,102.0,0.0,15.0,0.0,383.0,1075.0,1455.0,2747.0,2201.0,751.0,90.0,1538.0,1720.0,1865.0,1157.0,806.0,859.0,1975.0,6425.0,560.0,3415.0,1694.0,0.0,0.0,0.0,0.0
2,Edam Cheese,Dairy and Egg Products,357,28.57,24.99,1.43,0.00,0.0,89.0,18.670,731,0.44,188,30,825.0,243.0,0.0,1.54,0.5,0.24,0.0,1.43,41.00,247.0,418.0,17.818,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,536.0,973,3.75,0.036,0.011,14.5,0.0,0.0,0.0,0.037,0.389,0.082,0.281,0.076,0.0,16.0,0.0,16.0,16.0,15.4,0.0,242.0,11.0,0.0,0.0,0.0,0.0,0.5,20.0,2.3,0.0,0.0,8125.0,665.0,0.0,0.0,0.0,0.0,352.0,932.0,1308.0,2570.0,2660.0,721.0,255.0,1434.0,1457.0,1810.0,964.0,1034.0,764.0,1747.0,6150.0,486.0,3251.0,1547.0,0.0,0.0,0.0,0.0
3,Gouda Cheese,Dairy and Egg Products,356,27.44,24.94,2.22,2.22,0.0,114.0,17.614,700,0.24,121,29,563.0,165.0,0.0,1.54,0.5,0.24,0.0,2.22,41.46,394.0,263.0,20.007,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,546.0,819,3.90,0.036,0.011,14.5,0.0,0.0,0.0,0.030,0.334,0.063,0.340,0.080,0.0,21.0,0.0,21.0,21.0,15.4,0.0,164.0,10.0,0.0,0.0,0.0,0.0,0.5,20.0,2.3,0.0,0.0,7747.0,657.0,0.0,0.0,0.0,0.0,352.0,930.0,1306.0,2564.0,2654.0,719.0,254.0,1431.0,1454.0,1806.0,962.0,1032.0,762.0,1743.0,6137.0,485.0,3245.0,1544.0,0.0,0.0,0.0,0.0
4,Fontina Cheese,Dairy and Egg Products,389,31.14,25.60,1.55,1.55,0.0,116.0,19.196,550,0.23,64,14,913.0,261.0,0.0,1.68,0.6,0.27,0.0,1.55,37.92,790.0,864.0,16.477,0.000,0.0,0.0,0.0,0.0,0.0,0.0,

# Food substitute recommender function: A little more advanced

Issues with the basic function:
- There is an error when food name has parentheses
- Among the matches returned, we should exclude those that are only variations of the name entered, e.g. 'asparagus' should not return 'cooked asparagus'


In [ ]:
import re

In [101]:
def food_substitute_recommender_m(food_name: str, food_data: pd.DataFrame, top_matches: int):
    """
    This function takes a food and returns a list of ranked foods most similar to it
    food_name: the name of the food we want a substitute for
    data: the dataset containing the matches
    top_matches: how many similar food we want returned
    """
    
    # To solve the problem with food names that have parenthesis, we will remove all non alphanumeric chars from
    # both the food name we look up for, and the dataset column we are looking in
    
    food_name = re.sub('[^A-Za-z0-9]+', ' ', food_name).strip()
    
    match = food_data.loc[food_data['Name'].str.replace('[^A-Za-z0-9]+', ' ', regex=True).str.strip().str.contains(food_name,case=False)]
    
    if match.empty:
        raise Exception("The food was not found. Try another one")

    index_int = match.index.values.astype(int)[0]
    
    
    
    gower_dist = gower.gower_topn(food_data.iloc[index_int:index_int+1,2:], food_data.iloc[:,2:], n = top_matches+1)
    
    #gower_dist = gower.gower_topn(food_data.iloc[index_int:index_int+1,2:], food_data.iloc[:,2:], n = top_matches+1)
    
    most_similar_indeces = gower_dist['index'][1:top_matches+1].astype(int)
    
    print(f"""The {top_matches} most similar foods to {food_name} are: """)
    
    return food_data.filter(items = most_similar_indeces,axis=0).reset_index(drop=True)
    

In [105]:
food_substitute_recommender_m('Bacon (Pan-Fried)', food_data, 3)

Bacon Pan Fried
The 3 most similar foods to Bacon Pan Fried are: 


,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
0,Pork Chops (Lean),Meats,195,6.90,31.00,0.0,0.0,0.0,88.0,2.327,11,0.73,428,26,6.0,2.0,0.0,0.64,0.4,0.25,0.0,0.0,62.16,51.0,1075.0,16.543,0.038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,303.0,58,2.09,0.085,0.014,47.4,0.0,0.0,0.0,0.676,0.271,7.982,1.011,0.538,0.0,0.0,0.0,0.0,0.0,89.9,3.5,2.0,0.0,0.0,0.0,0.0,0.0,0.4,17.0,0.0,0.0,0.0,2785.0,1217.0,40.0,0.0,11.0,0.0,369.0,1363.0,1468.0,2552.0,2757.0,850.0,350.0,1288.0,1228.0,1568.0,1998.0,1261.0,1776.0,2900.0,4736.0,1401.0,1253.0,1295.0,108.0,0.0,0.0,0.0
1,Grilled Top Round Steak,Meats,162,3.77,30.09,0.0,0.0,0.0,86.0,1.556,18,3.25,435,17,9.0,3.0,0.0,2.28,0.0,0.32,0.0,0.0,64.28,23.0,250.0,16.207,0.191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0,75,5.15,0.066,0.005,30.8,0.0,0.0,0.0,0.089,0.328,8.975,0.497,0.882,0.0,5.0,0.0,5.0,5.0,90.1,15.4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.5,0.0,0.0,1794.0,310.0,6.0,3.0,13.0,1.0,402.0,1683.0,1670.0,3081.0,3443.0,951.0,359.0,1427.0,1337.0,1765.0,2402.0,1329.0,2138.0,3459.0,5840.0,1543.0,1504.0,1453.0,165.0,0.0,0.0,0.0
2,Pork Spare Ribs,Meats,397,30.30,29.06,0.0,0.0,0.0,121.0,11.120,47,1.85,320,24,10.0,3.0,0.0,1.08,2.6,0.34,0.0,0.0,40.42,110.0,2510.0,15.902,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,261.0,93,4.60,0.142,0.014,37.4,0.0,0.0,0.0,0.408,0.382,5.475,0.750,0.350,0.0,4.0,0.0,4.0,4.0,99.9,3.9,3.0,0.0,0.0,0.0,0.0,0.0,2.6,104.0,0.0,0.0,0.0,13480.0,2730.0,0.0,0.0,0.0,0.0,369.0,1327.0,1361.0,2332.0,2613.0,769.0,371.0,1160.0,1013.0,1577.0,1806.0,1161.0,1693.0,2695.0,4549.0,1380.0,1167.0,1200.0,0.0,0.0,0.0,0.0


In [109]:
food_data.sample(1)

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
241,Red Anjou Pears,Fruits,62,0.14,0.33,14.94,9.54,3.0,0.0,0.0,11,0.19,123,7,24.0,1.0,5.2,0.0,0.0,0.12,0.0,11.94,84.24,0.0,0.0,-2.28,0.0,0.0,0.0,0.26,2.74,6.48,0.0,0.0,0.0,0.0,0.0,13.0,1,0.13,0.07,0.056,0.1,0.0,0.0,0.0,0.012,0.028,0.162,0.053,0.039,0.0,7.0,0.0,7.0,0.0,5.1,0.0,0.0,14.0,0.0,0.0,53.0,0.0,0.0,0.0,4.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,10.0,17.0,15.0,2.0,1.0,10.0,1.0,15.0,9.0,1.0,12.0,94.0,27.0,11.0,19.0,14.0,0.0,0.0,0.0,0.0


In [111]:
food_substitute_recommender_m('Red Anjou Pears', food_data, 5)

Red Anjou Pears
The 5 most similar foods to Red Anjou Pears are: 


,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
0,Bartlett Pears,Fruits,63,0.16,0.39,15.01,9.69,3.1,0.0,0.000,9,0.19,101,6,28.0,1.0,4.4,0.0,0.0,0.12,0.0,11.91,84.14,0.0,0.0,-1.808,0.0,0.0,0.0,0.43,2.50,6.76,0.0,0.0,0.0,0.00,0.0,11.0,1,0.08,0.078,0.037,0.1,0.0,0.0,0.0,0.012,0.026,0.164,0.042,0.026,0.0,6.0,0.0,6.0,0.0,5.1,0.0,0.0,14.0,2.0,0.0,51.0,0.0,0.0,0.0,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,12.0,20.0,18.0,2.0,2.0,11.0,2.0,18.0,11.0,2.0,15.0,111.0,32.0,13.0,22.0,16.0,0.0,0.0,0.0,0.0
1,Pears,Fruits,57,0.14,0.36,15.23,9.75,3.1,0.0,0.022,9,0.18,116,7,25.0,1.0,4.3,0.0,0.0,0.12,0.0,12.13,83.96,1.0,93.0,-2.128,0.0,0.0,0.0,0.71,2.60,6.42,0.0,0.0,0.0,0.00,0.0,12.0,1,0.10,0.082,0.048,0.1,2.2,0.0,0.0,0.012,0.026,0.161,0.049,0.029,0.0,7.0,0.0,7.0,7.0,5.1,0.2,0.0,14.0,1.0,0.0,44.0,0.0,0.0,0.0,4.4,0.0,0.0,84.0,94.0,0.0,0.0,0.0,0.0,2.0,11.0,11.0,19.0,17.0,2.0,2.0,11.0,2.0,17.0,10.0,2.0,14.0,105.0,30.0,13.0,21.0,15.0,0.0,0.0,0.0,0.0
2,Green Anjou Pear,Fruits,66,0.10,0.44,15.79,9.73,3.1,0.0,0.000,11,0.24,127,7,25.0,1.0,4.4,0.0,0.0,0.12,0.0,12.69,83.31,0.0,0.0,-2.310,0.0,0.0,0.0,0.33,3.33,6.06,0.0,0.0,0.0,0.00,0.0,13.0,1,0.10,0.077,0.048,0.0,0.0,0.0,0.0,0.010,0.022,0.151,0.043,0.026,0.0,9.0,0.0,9.0,0.0,5.1,0.0,0.0,14.0,0.0,0.0,46.0,0.0,0.0,0.0,4.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,14.0,13.0,22.0,20.0,3.0,2.0,13.0,2.0,20.0,12.0,2.0,16.0,124.0,36.0,15.0,25.0,18.0,0.0,0.0,0.0,0.0
3,Red Delicious Apples,Fruits,59,0.20,0.27,14.06,10.48,2.3,0.0,0.000,6,0.11,104,5,55.0,3.0,0.0,0.0,0.0,0.24,0.0,11.76,85.33,0.0,0.0,-1.828,0.0,0.0,0.0,1.86,2.71,5.90,0.0,0.0,0.0,0.05,0.0,12.0,1,0.04,0.027,0.034,0.0,0.0,0.0,0.0,0.015,0.025,0.075,0.051,0.034,0.0,3.0,0.0,3.0,0.0,3.4,0.0,0.0,27.0,0.0,0.0,31.0,0.0,0.0,0.0,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,6.0,13.0,13.0,1.0,1.0,7.0,1.0,12.0,6.0,5.0,12.0,73.0,26.0,9.0,6.0,11.0,0.0,0.0,0.0,0.0
4,Apples,Fruits,52,0.17,0.26,13.81,10.39,2.4,0.0,0.028,6,0.12,107,5,54.0,3.0,4.6,0.0,0.0,0.18,0.0,11.41,85.56,9.0,43.0,-1.933,0.0,0.0,0.0,2.07,2.43,5.90,0.0,0.0,0.0,0.05,0.0,11.0,1,0.04,0.027,0.035,0.0,3.3,0.0,0.0,0.017,0.026,0.091,0.061,0.041,0.0,3.0,0.0,3.0,3.0,3.4,0.1,0.0,27.0,0.0,0.0,29.0,0.0,0.0,0.0,2.2,0.0,0.0,7.0,51.0,0.0,0.0,0.0,0.0,1.0,6.0,6.0,13.0,12.0,1.0,1.0,6.0,1.0,12.0,6.0,5.0,11.0,70.0,25.0,9.0,6.0,10.0,0.0,0.0,0.0,0.0
